In [23]:
import matplotlib.pyplot as plt
import torch
from torch.quasirandom import SobolEngine

import numpy as np
from alse.utils import identify_samples_which_satisfy_constraints, normalize, unnormalize
from mpl_toolkits import mplot3d
# from matplotlib import cm
from alse.utils import read_excel
from alse.alse import alse
import copy
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}

X, width, pow_cap, wth = read_excel("../test_data/8_12_data.xlsx",["P (W)", "V (mm/min)", "spot size (mm)"], ["width (mm)", "powder_cap", "wth"])
bounds = torch.tensor([[900, 600, 1], [2700, 2200, 4]])
constraints = [("gt", 2.5), ("gt", 0.65), ("gt", 6)]

algo = alse(X, bounds, [width, pow_cap, wth], constraints)

algo.initialize_model(["reg", "reg", "reg"])

algo.next_test_points(1)

tensor([[2626.6777, 2084.8844,    3.7802]], dtype=torch.float64)

In [24]:
%matplotlib widget

Normalized grid

In [25]:
from alse.accuracy import gen_test_point_for_accuracy
N = 100
Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(0, 1, N, **tkwargs), 
    torch.linspace(0, 1, N, **tkwargs),
    torch.linspace(0, 1, N, **tkwargs),
    indexing='xy',
)
xplt = torch.stack(
(
    torch.reshape(Xplt, (N**3,)),
    torch.reshape(Yplt, (N**3,)),
    torch.reshape(Zplt, (N**3,)),
),
dim=1,
)

In [26]:
a = algo.list_of_models[0](xplt).loc.detach() > 2.5
b = algo.list_of_models[1](xplt).loc.detach() > 0.65
c = algo.list_of_models[2](xplt).loc.detach() > 6
pos_overlap = (a & b & c).unsqueeze(-1)

Actual grid of parameter space

In [27]:
N = 100
Xplt, Yplt, Zplt = torch.meshgrid(
    torch.linspace(900, 2700, N, **tkwargs), 
    torch.linspace(600, 2200, N, **tkwargs),
    torch.linspace(1, 4, N, **tkwargs),
    indexing='xy',
)
xplt = torch.stack(
(
    torch.reshape(Xplt, (N**3,)),
    torch.reshape(Yplt, (N**3,)),
    torch.reshape(Zplt, (N**3,)),
),
dim=1,
)

In [28]:
a, b = gen_test_point_for_accuracy(pos_overlap, xplt, 12)

In bound test points

In [29]:
a

tensor([[2.2091e+03, 2.1838e+03, 3.1515e+00],
        [9.7273e+02, 1.8444e+03, 3.6061e+00],
        [1.7545e+03, 9.0707e+02, 3.9394e+00],
        [2.0818e+03, 1.5535e+03, 1.0909e+00],
        [2.1545e+03, 1.3434e+03, 3.0909e+00],
        [1.6636e+03, 1.2788e+03, 3.4848e+00]])

Out of bound test points

In [30]:
b

tensor([[1.5182e+03, 1.6343e+03, 1.3333e+00],
        [1.9000e+03, 1.5374e+03, 1.4848e+00],
        [1.7000e+03, 7.7778e+02, 1.4242e+00],
        [1.0818e+03, 1.4889e+03, 2.0909e+00],
        [9.1818e+02, 1.2465e+03, 1.6364e+00],
        [1.8455e+03, 6.3232e+02, 1.2121e+00]])

Ignore everything below

In [ ]:
# acq_val = algo.get_acq_val_grid(25, 3).round(decimals=4).detach()

Posterior

Surface Plot

In [ ]:
# N1, N2 = 100, 100
# Xplt, Yplt = torch.meshgrid(
#     torch.linspace(0, 1, N1, **tkwargs), 
#     torch.linspace(0, 1, N2, **tkwargs),
#     indexing='xy',
# )
# xplt = torch.stack(
# (
#     torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1],)),
#     torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1],)),
#     torch.ones(N1*N2)*0/100
# ),
# dim=1,
# )
# xp, yp = torch.meshgrid(
#     torch.linspace(900, 2700, N1, **tkwargs), 
#     torch.linspace(600, 2200, N2, **tkwargs),
#     indexing='xy',
# )

In [ ]:
# for i in range(120):
#     spot_size = round(i*3/120+1, 1)

#     xplt = torch.stack(
#     (
#         torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1],)),
#         torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1],)),
#         torch.ones(N1*N2)*i/120
#     ),
#     dim=1,
#     )
    
#     # predicted_width = algo.list_of_models[0](xplt).loc.detach().reshape(100,100)
#     # predicted_pow_cap = algo.list_of_models[1](xplt).loc.detach().reshape(100,100)
#     predicted_wth = algo.list_of_models[2](xplt).loc.detach().reshape(100,100)
#     fig = plt.figure(figsize=(10, 10))
#     ax = plt.axes(projection='3d')

#     ax.set_xlabel("Power")
#     ax.set_ylabel("Velocity")
#     # ax.set_zlabel("Predicted Width")
#     ax.set_zlabel("Predicted wth")
#     img = ax.plot_surface(xp, yp, predicted_wth, cmap='coolwarm')
#     ax.set_title(f'Spot size: {spot_size}mm')
#     ax.set_xlim(900, 2700)
#     ax.set_ylim(600, 2200)
#     ax.set_zlim(4, 24)

#     # cbar = fig.colorbar(img, fraction=0.025)
#     # cbar.set_label('Width', rotation=270)
#     # plt.show()
#     plt.savefig(f'./img/{i}.png', dpi=300)
#     plt.close()

Scatter plot

In [ ]:
# N = 5
# Xplt, Yplt, Zplt= torch.meshgrid(
#     torch.linspace(0, 1, N, **tkwargs), 
#     torch.linspace(0, 1, N, **tkwargs),
#     torch.linspace(0, 1, N, **tkwargs),
#     indexing='xy',
# )
# xplt = torch.stack(
#     (
#         torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1] * Xplt.shape[2],)),
#         torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1] * Yplt.shape[2],)),
#         torch.reshape(Zplt, (Zplt.shape[0] * Zplt.shape[1] * Zplt.shape[2],)),
#     ),
#     dim=1,
# )

In [ ]:
# predicted_width = algo.list_of_models[2](xplt).loc.detach().reshape(N,N,N)

In [ ]:
# N = 5
# Xplt, Yplt, Zplt= torch.meshgrid(
#     torch.linspace(900, 2700, N, **tkwargs), 
#     torch.linspace(600, 2200, N, **tkwargs),
#     torch.linspace(1, 4, N, **tkwargs),
#     indexing='xy',
# )

In [ ]:
# fig = plt.figure(figsize=(10, 10))
# ax = plt.axes(projection='3d')

# ax.set_xlabel("Power")
# ax.set_ylabel("Velocity")
# ax.set_zlabel("Spot Size")
# # img = ax.bar3d(X[:,0], X[:,1], 0, 20, 20, 1, shade=True)
# # img = ax.contourf3D(Xplt, Yplt, predicted_width, 800, cmap='coolwarm')
# # img = ax.scatter(Xplt, Yplt, Zplt, s=size_map)
# img = ax.scatter(Xplt, Yplt, Zplt, c=predicted_width, s=80)
# # ax.set_title(f'Spot size: {spot_size}mm')
# ax.set_xlim(900, 2700)
# ax.set_ylim(600, 2200)
# ax.set_zlim(1, 4)

# cbar = fig.colorbar(img, fraction=0.025)
# cbar.set_label('wth', rotation=270)
# # plt.show()

In [ ]:
# size_map = (predicted_width-4.9)*40

In [ ]:
# fig = plt.figure(figsize=(10, 10))
# ax = plt.axes(projection='3d')

# ax.set_xlabel("Power")
# ax.set_ylabel("Velocity")
# ax.set_zlabel("Spot Size")
# # img = ax.bar3d(X[:,0], X[:,1], 0, 20, 20, 1, shade=True)
# # img = ax.contourf3D(Xplt, Yplt, predicted_width, 800, cmap='coolwarm')
# img = ax.scatter(Xplt, Yplt, Zplt, s=size_map, c='k')
# # img = ax.scatter(Xplt, Yplt, Zplt, c=predicted_width, s=80)
# # ax.set_title(f'Spot size: {spot_size}mm')
# ax.set_xlim(900, 2700)
# ax.set_ylim(600, 2200)
# ax.set_zlim(1, 4)

# # cbar = fig.colorbar(img, fraction=0.025)
# # cbar.set_label('Width', rotation=270)
# # plt.show()